# AI Crash Course: Supervised Learning

In this crash course, we will introduce RDKit, a library for cheminformatics and one of the most popular machine learning frameworks, PyTorch. Both libraries are open-source. PyTorch is based on the Torch library and was originally developed by Meta AI, but is now under the Linux Foundation umbrella.

We start by installing all the required libraries for this exercise. This is done with `pip`, the package installer for Python: https://pypi.org/project/pip/

In [ ]:
# Install required libraries
! pip install pytorch-lightning rdkit torch gdown

Here, we load all the packages we need for the practical part.

In [ ]:
# Load all packages we will need
## Handling random seeds
import random as rd
## Download data from Google Drive
import gdown as gd
## Handling data
import numpy as np
import pandas as pd
## Cheminformatics
import rdkit.Chem as rdc
import rdkit.Chem.Descriptors as rdcd
## Capture warnings to suppress them
import sys
import io
## Data science utilities
import scipy.stats as sps
import sklearn.model_selection as skms
import sklearn.preprocessing as skpp
## Machine learning backend
import pytorch_lightning as pl
import pytorch_lightning.loggers as pll
import torch as tc
import torch.optim as tco
import torch.nn.functional as tcnf
import torch.utils.data as tcud
## Plot results
import matplotlib.pyplot as plt

## Artificial Neural Network for pKa Prediction with PyTorch

In the following section, we will implement an artificial neural network for the prediction of the pKa of small acidic organic molecules in water. This is based on the pKa dataset provided with DataWarrior: https://openmolecules.org/datawarrior/download.html

As the dataset can only be downloaded from the website together with the program, the dataset alone was uploaded to Google Drive for this crash course.

In [ ]:
# Download pKa dataset from Google Drive
google_file_id = "10M_x9pHD6-t35Au-l3p-lEqT6y44xbG-"
output_filename = "data.txt"
gd.download(f"https://drive.google.com/uc?id={google_file_id}", output_filename)

Next, we load the dataset into python and create a new clean dataframe that only contains the data of interest. The dataset contains pKa values for both acids and bases.

In [ ]:
# Load database from file
database = pd.read_csv('./data.txt', delimiter=',')
print(database)

In this crash course, we make the life of the machine learning model easier by only training a predictive model for the pKa values of organic acids. Feel free to go back here and also perform this exercise for organic bases by changing `'acidic'` to `' basic'` in line 3. At the end, we extract the pKa range from the remaining datapoints.

In [ ]:
# Select subset of organic acids
database = database.loc[database['basicOrAcidic'] == 'acidic']

# Read out columns of interest
data = pd.DataFrame()
data['smiles'] = database['Smiles'].values
data['pka'] = database['pKa'].values
print(data)

# Extract data range
pka_range = [data['pka'].min(), data['pka'].max()]

Now, we are interested to create a first probability density that represents this dataset. That is, we want to know how frequent specific pKa values appear. This allows finding target value ranges that are underrepresented in a dataset. We do this via kernel density estimation, which is implemented in `scikit-learn`.

In [ ]:
# Create distribution density  of dataset
density = sps.gaussian_kde(data['pka'])
density_points = np.linspace(pka_range[0], pka_range[1], 200)

# Plot distribution density of dataset
plt.plot(density_points, density(density_points))
plt.xlabel('pKa')
plt.ylabel('Probability Density')
plt.show()

Based on this plot, it is clear that pKa values outside the range [0.0, 15.0] are very rare. To avoid training our model on these rare datapoints, we limit the range of the model to the range [0.0, 15.0]. This step is not necessary, but will lead to a machine learning model with better prediction performance within this pKa value range.

In [ ]:
# Remove isolated datapoint of very high or very low pKa
data = data.loc[data['pka'] > 0.]
data = data.loc[data['pka'] < 15.]
print(data)

# Extract data range
pka_range = [data['pka'].min(), data['pka'].max()]

Now, we use RDKit to compute features to be used as input for the artificial neural network. Here, we use simple molecular descriptors that are implemented in RDKit.

In [ ]:
# Define objects to capture standard error output from rdkit
original_stderr = sys.stderr
sys.stderr = io.StringIO()

# Create mol objects from SMILES
data['mols'] = [rdc.MolFromSmiles(si) for si in data['smiles'].values.tolist()]

# Compute all available descriptors in RDKit (it takes some time)
features = pd.DataFrame([rdcd.CalcMolDescriptors(mi) for mi in data['mols'].values.tolist()])
print(features)

# Capture standard error output and suppress rdkit warnings
captured_output = sys.stderr.getvalue()
sys.stderr = original_stderr

Next, we need to do some data cleaning. Sometimes, RDKit cannot properly process a SMILES string of a molecule. This leads to a molecule with no molecular descriptors. We cannot use such data for either training or model evaluation. Hence, we simply remove these molecules from our dataset. This is done in the following code cell. Additionally, we also remove a few features that frequently do not lead to numberical values but rather assign `NaN`, which stands for "not a number".

In [ ]:
# Remove molecules with NaN descriptors
data = data.drop(features[features.isnull().all(axis=1)].index)
features = features.drop(features[features.isnull().all(axis=1)].index)

# Remove descriptor columns with at least one value being not a number (NaN)
features = features.dropna(axis='columns')
print(features)

# Get a list of all computed descriptors
list_of_features = features.columns.values.tolist()

Now, let us visualize a few of the molecules in a grid.

In [ ]:
# Visualize a few molecules in a grid
# Select random indices
indices = rd.sample(range(len(data)), 6)

# Select molecules based on random indices
mol_list = [data['mols'].iloc[ni] for ni in indices]

# Now we create the GridImage
grid = rdc.Draw.MolsToGridImage(mol_list, subImgSize=(300,300)) # create the image

grid # visualize your molecules

For being able to obtain the exact same results when rerunning your code, it is important to set random seeds. Otherwise, as many of the methods implemented rely on stochastic processes, you will obtain different results every time you run the code and this is not ideal especially when you are in the implementation and debugging stage of a machine learning project.

In [ ]:
# Random seeds for reproducibility
tc.manual_seed(0)
tc.cuda.manual_seed(0)
np.random.seed(0)
rd.seed(0)
tc.use_deterministic_algorithms(True)

### Training Workflow
The training of supervised learning models follows the general procedure depicted in the following figure.

workflow_trimmed.svg

Next, we prepare the data to be used for training, validation, and holdout. We start by splitting the data and then we scale it based on the training data. Scaling is particularly important to ensure that training proceeds without huge jumps and it also makes sure that all features are on the same scale.

In [ ]:
# Split dataset into training, validation and holdout data (70/20/10)
features_train, features_validation_holdout, data_train, data_validation_holdout = skms.train_test_split(features, data, test_size=0.30, random_state=0) # initial data split
features_validation, features_holdout, data_validation, data_holdout = skms.train_test_split(features_validation_holdout, data_validation_holdout, test_size=0.33, random_state=0) # split further

# Scale features
scaler_features = skpp.MinMaxScaler()
scaler_features.fit(features_train.values) # it is important to scale based on the training data, not based on the full data
features_train[features_train.columns[:]] = scaler_features.transform(features_train.values)
features_validation[features_validation.columns[:]] = scaler_features.transform(features_validation.values)
features_holdout[features_holdout.columns[:]] = scaler_features.transform(features_holdout.values)
print(features_train)

# Scale target values
scaler_targets = skpp.StandardScaler()
scaler_targets.fit(data_train['pka'].values.reshape(-1, 1))
targets_train = scaler_targets.transform(data_train['pka'].values.reshape(-1, 1))
targets_validation = scaler_targets.transform(data_validation['pka'].values.reshape(-1, 1))
targets_holdout = scaler_targets.transform(data_holdout['pka'].values.reshape(-1, 1))

Next, we need to define a class for our artificial neural network. In PyTorch Lightning, this is the way to define any neural network. In case your knowledge about classes in Python is rusty, have a look at the brief tutorial provided in the practicals from week 2.

In [ ]:
class NeuralNetwork(pl.LightningModule):
  def __init__(self, dataset_train, dataset_validation, dataset_holdout, input_size, hidden_size, hidden_layers, activation_function, dropout_rate, batch_size=100, learning_rate=1e-3):
    # Run the initialization from the pl.LightningModule class
    super().__init__()

    # Define class variables
    self.dataset_train = dataset_train
    self.dataset_validation = dataset_validation
    self.dataset_holdout = dataset_holdout
    self.batch_size = batch_size
    self.learning_rate = learning_rate

    # Define neural network architecture
    self.model = tc.nn.Sequential(
      tc.nn.Linear(input_size, hidden_size), # this is a fully connected linear transformation with the input features
      activation_function, # this is the activation function to be used that is applied after the linear transformation
      tc.nn.Dropout(p=dropout_rate), # this implements dropout for this layer by stochastically dropping a certain percentage of the nodes
    )
    if hidden_layers > 1:
      for li in range(hidden_layers - 1):
        self.model.append(tc.nn.Linear(hidden_size, hidden_size))
        self.model.append(activation_function)
        self.model.append(tc.nn.Dropout(p=dropout_rate))
    self.model.append(tc.nn.Linear(hidden_size, 1)) # this is the final layer that provides one number as output, the property of interest
    return

  def training_step(self, batch, batch_index):
    # Define training loss computation.
    y_in, y_out = batch
    y_hat = self.model(y_in)
    loss = tcnf.mse_loss(y_hat, y_out) # mean-squared error loss
    self.log("Training MSE", loss) # this makes sure that intermediate results are logged during training
    return loss

  def validation_step(self, batch, batch_index):
    # Define validation loss computation.
    y_in, y_out = batch
    y_hat = self.model(y_in)
    loss = tcnf.mse_loss(y_hat, y_out) # mean-squared error loss
    self.log("Validation MSE", loss) # this makes sure that intermediate results are logged during training
    return

  def test_step(self, batch, batch_index):
    # Define holdout loss computation.
    y_in, y_out = batch
    y_hat = self.model(y_in)
    loss = tcnf.mse_loss(y_hat, y_out) # mean-squared error loss
    self.log("Holdout MSE", loss) # this makes sure that intermediate results are logged during training
    return

  def configure_optimizers(self):
    # Configure the optimization algorithm.
    optimizer = tco.Adam(self.parameters(), lr=self.learning_rate)
    return optimizer

  def forward(self, y_in):
    # Define forward pass
    return self.model(y_in).flatten()

  def train_dataloader(self):
    return tcud.DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return tcud.DataLoader(self.dataset_validation, batch_size=self.batch_size, shuffle=False)

  def test_dataloader(self):
    return tcud.DataLoader(self.dataset_holdout, batch_size=self.batch_size, shuffle=False)

In addition to defining a class for the model architecture, we also need to define a class for our dataset.

In [ ]:
class PKADataset(tcud.Dataset):
  def __init__(self, X, y):
    # Define class variables
    self.X = X
    self.y = y

  def __len__(self):
    # Define output of len(self) function evaluation
    return self.X.shape[0]

  def __getitem__(self, idx):
    # Define function returning the dataset contents as tensor
    if tc.is_tensor(idx):
        idx = idx.tolist()
    X_idx = tc.as_tensor(self.X[idx].astype(np.float32))
    y_idx = tc.as_tensor(self.y[idx].astype(np.float32).reshape(-1))
    return X_idx, y_idx

Furthermore, to store the training and validation loss during the training process, we need to define a custom callback for collecting training metrics.

In [ ]:
class CollectMetricsCallback(pl.Callback):
  def __init__(self):
    # Define class variables
    self.training_mse_list = list()
    self.training_epoch_list = list()
    self.validation_mse_list = list()
    self.validation_epoch_list = list()
    self.training_history = pd.DataFrame()

  def on_train_epoch_end(self, trainer, pl_module):
    # Check if current Training MSE exists
    if trainer.callback_metrics.get('Training MSE') is not None:
      # Append current epoch number and Training MSE to list
      self.training_epoch_list.append(trainer.current_epoch)
      self.training_mse_list.append(trainer.callback_metrics.get('Training MSE').item())

  def on_validation_epoch_end(self, trainer, pl_module):
    # Check if current Validation MSE exists
    if trainer.callback_metrics.get('Validation MSE') is not None:
      # Prevent having two entries for epoch 0
      if len(self.validation_mse_list) == 1 and trainer.current_epoch == 0:
        # Overwrite the original Validation MSE value for epoch 0 with the latest one
        self.validation_mse_list[0] = trainer.callback_metrics['Validation MSE'].item()

      else:
        # Append current epoch number and Validation MSE to list
        self.validation_epoch_list.append(trainer.current_epoch)
        self.validation_mse_list.append(trainer.callback_metrics['Validation MSE'].item())

  def on_train_end(self, trainer, pl_module):
    # Collect all results in one pandas dataframe
    self.training_history['Training Epoch'] = self.training_epoch_list
    self.training_history['Training MSE'] = self.training_mse_list
    self.training_history['Validation Epoch'] = self.validation_epoch_list
    self.training_history['Validation MSE'] = self.validation_mse_list

Having defined classes for our data and the model architecture, we can now finally proceed to actually training a model. Hence, we first appropriate objects for our data, then define the hyperparameters in a dictionary so they are all available in one object in a user-friendly manner. Then, we can finally initialize the `NeuralNetwork` object. Before training, we also need to define a logger object that ensures that data during training is collected which we can then use for following the training progress. Here, we make use of the `CSVLogger` that is implemented in PyTorch Lightning. This logger stores all the information locally. There are other alternatives, but they require to setup accounts on dedicated websites or set up local services for data inspection. Furthermore, to make accessing the training and validation losses more convenient, we initialize a custom `CollectMetricsCallback` called `metrics_callback`. Finally, we define a `Trainer` which allows specifying training parameters. Only then, we can perform the actual model training with the `trainer.fit()` function. Importantly, executing this code cell will take some time but you can follow the process as the output is constantly updated.

In [ ]:
# Create data instances
dataset_train = PKADataset(features_train.values, targets_train)
dataset_validation = PKADataset(features_validation.values, targets_validation)
dataset_holdout = PKADataset(features_holdout.values, targets_holdout)

# Dictionary of hyperparameters
hyperparameters = {
  'hidden_size': 256,
  'hidden_layers': 3,
  'activation_function': tc.nn.ReLU(),
  'dropout_rate': 0.2,
  'batch_size': 100,
  'learning_rate': 1e-3,
  'maximum_epochs': 100,
}

# Create an instance of our neural network
neural_network = NeuralNetwork(
  dataset_train = dataset_train,
  dataset_validation = dataset_validation,
  dataset_holdout = dataset_holdout,
  input_size = features_train.shape[1],
  hidden_size = hyperparameters['hidden_size'],
  hidden_layers = hyperparameters['hidden_layers'],
  activation_function = hyperparameters['activation_function'],
  dropout_rate = hyperparameters['dropout_rate'],
  batch_size = hyperparameters['batch_size'],
  learning_rate = hyperparameters['learning_rate'],
)

# Define data logger
logger = pll.CSVLogger('logs', name='pka')

# Define metrics callback for collecting validation and holdout loss
metrics_callback = CollectMetricsCallback()

# Define trainer for neural network
trainer = pl.Trainer(
    max_epochs = hyperparameters["maximum_epochs"],
    logger = logger,
    log_every_n_steps = 1,
    callbacks=[metrics_callback]
)

# Training a model
trainer.fit(model=neural_network)

# Log the hyperparameters used
logger.log_hyperparams(hyperparameters)

# Determine holdout performance (final model)
results = trainer.test(ckpt_path='best')
holdout_mse = results[0]["Holdout MSE"]
print(f"\nANN model performance: MSE on holdout set = {holdout_mse:.4f}\n")

After having performed training, we are interested to inspect the training progress. This data is available via two separate channels.

### Metrics Callback
This is the most straightforward way to access both the training and validation loss for the entire training process. Hence, this is the channel that we will make use of in the following code cell. The `pd.DataFrame` accessible via `metrics_callback.training_history` contains all the data we need. This is the case because we defined a custom metrics callback that collects these losses during training.

### CSV Logger
Alternatively, the corresponding data has been logged via the `CSVLogger`. This logger stores the data on the local file system. If you click on the folder icon on the left command bar of Google Colab, and open the folder `logs`, you should find a folder named as the `name` we defined as parameter when creating the logger instance. That means that it should be called `pka`. Inside that folder, there should be a folder named `version_0`. This is the case if you ran the above code once. If you run it multiple times, it will create additional folders and the number will increment. Therefore, you need to make sure to also change the corresponding folder name accordingly. The folder `version_0` now contains the data of interest. In the file `metrics.csv`, we find the training and validation loss that we need for following the training process. In the file `hparams.yaml`, we find the hyperparameters of the model that we used. In the folder `checkpoints`, there is a `ckpt` file that contains all the trained model parameters and hyperparameters so that somebody else could use it for predictions. For now, we would only need to make use of the `metrics.csv` file.

In [ ]:
# Plot model prediction against actual values for both the train data and the test data
plt.plot(metrics_callback.training_history['Training Epoch'], metrics_callback.training_history['Training MSE'], label='Train')
plt.plot(metrics_callback.training_history['Validation Epoch'], metrics_callback.training_history['Validation MSE'], label='Validation')
plt.legend()
plt.xlabel('Training Epoch')
plt.ylabel('Mean Squared Error')
plt.show()

As you can see, training proceeds quite well, both for the training loss and the validation loss. It is quite normal that the validation loss is somewhat higher than the training loss. Next, we also want to test the prediction performance of our training artificial neural network and compare it to the experimental values. This is done in the following code cell.

In [ ]:
# Set model to evaluation mode
neural_network.eval() # this is especially important when your model has dropout layers as they are deactivated after training

# Make predictions with the models
predictions_train = scaler_targets.inverse_transform(neural_network(dataset_train[:][0]).detach().numpy().reshape(-1, 1)) # this seemingly complicated string of operations converts a tensor object to a numpy array and then reshapes it
predictions_validation = scaler_targets.inverse_transform(neural_network(dataset_validation[:][0]).detach().numpy().reshape(-1, 1)) # data from a PyTorch Dataset object is stored as tensor allowing for efficient linear algebra
predictions_holdout = scaler_targets.inverse_transform(neural_network(dataset_holdout[:][0]).detach().numpy().reshape(-1, 1)) # the reshape operation is required for the inverse_transform of the scaler

# Plot model prediction against actual values for both the train data and the test data
plt.scatter(predictions_train, data_train['pka'].values, label='Train')
plt.scatter(predictions_validation, data_validation['pka'].values, label='Validation')
plt.scatter(predictions_holdout, data_holdout['pka'].values, label='Holdout')
plt.plot(pka_range , pka_range, linestyle='--', color='black', label='Ideal')
plt.legend()
plt.xlabel('Predicted pKa')
plt.ylabel('Experimental pKa')
plt.show()

It took some code cells, but now you know how to define a model, train it, and inspect the model accuracy. As a last step, it is good too look at some model predictions in the holdout set by visualizing some of the corresponding molecules together with their predicted and experimental pKa values. For some molecules, you will find that our model shows excellent predictions. For others, you will find substantial errors. This is to be expected after the plot we generated in the previous code cell.

In [ ]:
# Visualize a few molecules from the holdout set with their predictions in a grid
# Select random indices
indices = rd.sample(range(len(data_holdout)), 6)

# Select molecules based on random indices
mol_list = [data_holdout['mols'].iloc[ni] for ni in indices]

# Now we create the GridImage
grid = rdc.Draw.MolsToGridImage(mol_list, subImgSize=(300,300), legends = [f"Prediction: {predictions_holdout[ni][0]:.2f}\nExperiment: {data_holdout['pka'].values[ni]}" for ni in indices]) # create the image

grid # visualize your molecules